In [ ]:
!pip install spotipy
import pandas as pd
import numpy as np
import timeit
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials 
from google.colab import files

In [ ]:
client_id = '***'
client_secret = '***'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 

In [29]:
data = pd.read_csv('big5_mft_tracks.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)
data

,artist_name,song_title,preview_url,track_id,artist_id,track_popularity,processed_artist_name
0,12th Planet,Follow,https://p.scdn.co/mp3-preview/ef07dc598b117870...,1HJW2w61lh3XcsnfnOR9p4,3V1h3kAdiVDBiwlY2i6dJz,40.0,12th planet
1,12th Planet,Swamplex Terrestrial,https://p.scdn.co/mp3-preview/bfe1fffd48350137...,35Qf9mP6LoopjOVwgPAKoY,3V1h3kAdiVDBiwlY2i6dJz,26.0,12th planet
2,12th Planet,Supernova,https://p.scdn.co/mp3-preview/a492095a74937ed7...,5Yu2XAETE0p1dLzbgoX3WN,3V1h3kAdiVDBiwlY2i6dJz,24.0,12th planet
3,12th Planet,Let It Bang,https://p.scdn.co/mp3-preview/5cb107204eaa46ec...,2A7C2pIp1N0sgOi0aM6z3g,3V1h3kAdiVDBiwlY2i6dJz,20.0,12th planet
4,12th Planet,Let Us Prey,https://p.scdn.co/mp3-preview/15a9234eff6e1126...,5TMYoiWElQzD4UsTvtX8kx,3V1h3kAdiVDBiwlY2i6dJz,15.0,12th planet
...,...,...,...,...,...,...,...
11175,Twenty One Pilots,Stressed Out,https://p.scdn.co/mp3-preview/4d82e9c363b235d0...,3CRDbSIZ4r5MsZ0YwxuEkn,3YQKmKGau1PzlVlkL1iodx,85.0,twenty one pilots
11176,Twenty One Pilots,Ride,https://p.scdn.co/mp3-preview/c6011d475d777067...,2Z8WuEywRWYTKe1NybPQEW,3YQKmKGau1PzlVlkL1iodx,82.0,twenty one pilots
11177,Twenty One Pilots,Heathens,https://p.scdn.co/mp3-preview/1eec0afcc9b80870...,6i0V12jOa3mr6uu4WYhUBr,3YQKmKGau1PzlVlkL1iodx,81.0,twenty one pilots
11178,Twenty One Pilots,Chlorine,https://p.scdn.co/mp3-preview/393603fe15a15ca5...,23OXdR7YuUBVWh5hSnYJau,3YQKmKGau1PzlVlkL1iodx,76.0,twenty one pilots


In [47]:
# get audio features
start = timeit.default_timer()
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(data['track_id']),batchsize):
    batch = data['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks where no audio features were available:',None_counter)

stop = timeit.default_timer()
print ('Time to run this code (in seconds):',stop - start)

Number of tracks where no audio features were available: 1
Time to run this code (in seconds): 11.489472455000396


In [48]:
features_df = pd.DataFrame.from_dict(rows,orient='columns')
features_df = features_df.drop(columns = ['type', 'uri', 'track_href', 'analysis_url', 'duration_ms'])
features_df = features_df.rename(columns={'id':'track_id'})
features_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_id,time_signature
0,0.533,0.938,4,-2.762,0,0.0535,0.027400,0.004720,0.1910,0.1180,149.998,1HJW2w61lh3XcsnfnOR9p4,4
1,0.515,0.974,1,-2.461,1,0.5060,0.000533,0.016900,0.1210,0.0558,75.032,35Qf9mP6LoopjOVwgPAKoY,4
2,0.589,0.992,11,-1.958,0,0.4220,0.042800,0.000000,0.6610,0.5100,150.105,5Yu2XAETE0p1dLzbgoX3WN,4
3,0.581,0.960,1,-2.478,1,0.2200,0.000871,0.003460,0.1310,0.1550,149.923,2A7C2pIp1N0sgOi0aM6z3g,4
4,0.345,0.896,9,-2.695,1,0.3740,0.008430,0.031200,0.1280,0.1490,75.897,5TMYoiWElQzD4UsTvtX8kx,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11174,0.734,0.637,4,-5.677,0,0.1410,0.046200,0.000023,0.0602,0.6480,169.977,3CRDbSIZ4r5MsZ0YwxuEkn,4
11175,0.645,0.713,6,-5.355,1,0.0393,0.008350,0.000000,0.1130,0.5660,74.989,2Z8WuEywRWYTKe1NybPQEW,4
11176,0.732,0.396,4,-9.348,0,0.0286,0.084100,0.000036,0.1050,0.5480,90.024,6i0V12jOa3mr6uu4WYhUBr,4
11177,0.609,0.674,10,-7.388,0,0.0548,0.073500,0.060000,0.3450,0.3150,90.009,23OXdR7YuUBVWh5hSnYJau,4


In [49]:
final_df = pd.merge(data, features_df, how="left")
final_df

,artist_name,song_title,preview_url,track_id,artist_id,track_popularity,processed_artist_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,12th Planet,Follow,https://p.scdn.co/mp3-preview/ef07dc598b117870...,1HJW2w61lh3XcsnfnOR9p4,3V1h3kAdiVDBiwlY2i6dJz,40.0,12th planet,0.533,0.938,4.0,-2.762,0.0,0.0535,0.027400,0.004720,0.1910,0.1180,149.998,4.0
1,12th Planet,Swamplex Terrestrial,https://p.scdn.co/mp3-preview/bfe1fffd48350137...,35Qf9mP6LoopjOVwgPAKoY,3V1h3kAdiVDBiwlY2i6dJz,26.0,12th planet,0.515,0.974,1.0,-2.461,1.0,0.5060,0.000533,0.016900,0.1210,0.0558,75.032,4.0
2,12th Planet,Supernova,https://p.scdn.co/mp3-preview/a492095a74937ed7...,5Yu2XAETE0p1dLzbgoX3WN,3V1h3kAdiVDBiwlY2i6dJz,24.0,12th planet,0.589,0.992,11.0,-1.958,0.0,0.4220,0.042800,0.000000,0.6610,0.5100,150.105,4.0
3,12th Planet,Let It Bang,https://p.scdn.co/mp3-preview/5cb107204eaa46ec...,2A7C2pIp1N0sgOi0aM6z3g,3V1h3kAdiVDBiwlY2i6dJz,20.0,12th planet,0.581,0.960,1.0,-2.478,1.0,0.2200,0.000871,0.003460,0.1310,0.1550,149.923,4.0
4,12th Planet,Let Us Prey,https://p.scdn.co/mp3-preview/15a9234eff6e1126...,5TMYoiWElQzD4UsTvtX8kx,3V1h3kAdiVDBiwlY2i6dJz,15.0,12th planet,0.345,0.896,9.0,-2.695,1.0,0.3740,0.008430,0.031200,0.1280,0.1490,75.897,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11175,Twenty One Pilots,Stressed Out,https://p.scdn.co/mp3-preview/4d82e9c363b235d0...,3CRDbSIZ4r5MsZ0YwxuEkn,3YQKmKGau1PzlVlkL1iodx,85.0,twenty one pilots,0.734,0.637,4.0,-5.677,0.0,0.1410,0.046200,0.000023,0.0602,0.6480,169.977,4.0
11176,Twenty One Pilots,Ride,https://p.scdn.co/mp3-preview/c6011d475d777067...,2Z8WuEywRWYTKe1NybPQEW,3YQKmKGau1PzlVlkL1iodx,82.0,twenty one pilots,0.645,0.713,6.0,-5.355,1.0,0.0393,0.008350,0.000000,0.1130,0.5660,74.989,4.0
11177,Twenty One Pilots,Heathens,https://p.scdn.co/mp3-preview/1eec0afcc9b80870...,6i0V12jOa3mr6uu4WYhUBr,3YQKmKGau1PzlVlkL1iodx,81.0,twenty one pilots,0.732,0.396,4.0,-9.348,0.0,0.0286,0.084100,0.000036,0.1050,0.5480,90.024,4.0
11178,Twenty One Pilots,Chlorine,https://p.scdn.co/mp3-preview/393603fe15a15ca5...,23OXdR7YuUBVWh5hSnYJau,3YQKmKGau1PzlVlkL1iodx,76.0,twenty one pilots,0.609,0.674,10.0,-7.388,0.0,0.0548,0.073500,0.060000,0.3450,0.3150,90.009,4.0


In [50]:
df_nan = final_df[final_df.isna().any(axis=1)]
df_nan

,artist_name,song_title,preview_url,track_id,artist_id,track_popularity,processed_artist_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
5602,Natasha Bedingfield,These Words,https://p.scdn.co/mp3-preview/8ed0b0fda38d49e5...,6MFQeWtk7kxWGydnJB2y36,7o95ZoZt5ZYn31e9z1Hc0a,65.0,natasha bedingfield,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
final_df.to_csv('spotify_features.csv')
files.download('spotify_features.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>